In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
sns.set(color_codes=True)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import random



from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 

In [2]:
df = pd.read_csv('data_cl.csv')
df

,restaurant_id,menu,menu_final
0,1,"['extra', 'large', 'meat', 'lovers', 'whole', ...","['extra', 'large', 'meat', 'lover', 'whole', '..."
1,2,"['j', '’', 'ti', '`', '’', 'z', 'smoothie', 't...","['j', 'ti', 'z', 'smoothie', 'tropical', 'frui..."
2,3,"['chicken', 'bacon', 'ranch', 'cheesesteak', '...","['chicken', 'bacon', 'ranch', 'cheesesteak', '..."
3,4,"['chicken', 'bacon', 'artichoke', '-', 'keto',...","['chicken', 'bacon', 'artichoke', 'keto', 'fri..."
4,5,"['full', 'sausage', '(', '2', 'pcs', ')', 'wit...","['full', 'sausage', 'pc', 'egg', 'piece', 'dou..."
...,...,...,...
60922,63465,"['breakfast', 'sampler', 'two', 'eggs', '*', '...","['breakfast', 'sampler', 'two', 'egg', 'way', ..."
60923,63466,"['ready', 'rabbit', 'blue', 'curacao', 'non', ...","['ready', 'rabbit', 'blue', 'curacao', 'non', ..."
60924,63467,"['heineken', 'non-alcoholic', '0.0', '6x', '11...","['heineken', 'can', 'master', 'brewer', 'start..."
60925,63468,"['3', 'piece', 'combo', 'three', 'pieces', 'of...","['piece', 'combo', 'three', 'piece', 'chicken'..."


In [3]:
import pickle

# Load the lemmatizer from the file
with open('wordnet_lemmatizer.pkl', 'rb') as f:
    lemmatizer = pickle.load(f)


In [4]:
restaurant_df = pd.read_csv('restaurants.csv')
restaurant_df = restaurant_df[['id', 'name']]
restaurant_df

,id,name
0,1,PJ Fresh (224 Daniel Payne Drive)
1,2,J' ti`'z Smoothie-N-Coffee Bar
2,3,Philly Fresh Cheesesteaks (541-B Graymont Ave)
3,4,Papa Murphy's (1580 Montgomery Highway)
4,5,Nelson Brothers Cafe (17th St N)
...,...,...
63464,63465,IHOP (4321 N 10th St)
63465,63466,Feldman's - N 10th Street
63466,63467,Feldman's - N 23rd St
63467,63468,Church's Chicken (620 S. 10th Street)


In [5]:
merged_df = pd.merge(df, restaurant_df, left_on='restaurant_id', right_on='id', how='left')

# Drop the redundant 'id' column
merged_df.drop(columns=['id'], inplace=True)

# Rename the column to 'restaurant_name'
merged_df.rename(columns={'name': 'restaurant_name'}, inplace=True)

merged_df

,restaurant_id,menu,menu_final,restaurant_name
0,1,"['extra', 'large', 'meat', 'lovers', 'whole', ...","['extra', 'large', 'meat', 'lover', 'whole', '...",PJ Fresh (224 Daniel Payne Drive)
1,2,"['j', '’', 'ti', '`', '’', 'z', 'smoothie', 't...","['j', 'ti', 'z', 'smoothie', 'tropical', 'frui...",J' ti`'z Smoothie-N-Coffee Bar
2,3,"['chicken', 'bacon', 'ranch', 'cheesesteak', '...","['chicken', 'bacon', 'ranch', 'cheesesteak', '...",Philly Fresh Cheesesteaks (541-B Graymont Ave)
3,4,"['chicken', 'bacon', 'artichoke', '-', 'keto',...","['chicken', 'bacon', 'artichoke', 'keto', 'fri...",Papa Murphy's (1580 Montgomery Highway)
4,5,"['full', 'sausage', '(', '2', 'pcs', ')', 'wit...","['full', 'sausage', 'pc', 'egg', 'piece', 'dou...",Nelson Brothers Cafe (17th St N)
...,...,...,...,...
60922,63465,"['breakfast', 'sampler', 'two', 'eggs', '*', '...","['breakfast', 'sampler', 'two', 'egg', 'way', ...",IHOP (4321 N 10th St)
60923,63466,"['ready', 'rabbit', 'blue', 'curacao', 'non', ...","['ready', 'rabbit', 'blue', 'curacao', 'non', ...",Feldman's - N 10th Street
60924,63467,"['heineken', 'non-alcoholic', '0.0', '6x', '11...","['heineken', 'can', 'master', 'brewer', 'start...",Feldman's - N 23rd St
60925,63468,"['3', 'piece', 'combo', 'three', 'pieces', 'of...","['piece', 'combo', 'three', 'piece', 'chicken'...",Church's Chicken (620 S. 10th Street)


In [37]:
merged_df.set_index(pd.Index(range(len(merged_df))), inplace=True)
merged_df.index.name = 'restaurant_id'
merged_df

,menu,menu_final,restaurant_name
restaurant_id,,,
0,"['extra', 'large', 'meat', 'lovers', 'whole', ...","['extra', 'large', 'meat', 'lover', 'whole', '...",PJ Fresh (224 Daniel Payne Drive)
1,"['j', '’', 'ti', '`', '’', 'z', 'smoothie', 't...","['j', 'ti', 'z', 'smoothie', 'tropical', 'frui...",J' ti`'z Smoothie-N-Coffee Bar
2,"['chicken', 'bacon', 'ranch', 'cheesesteak', '...","['chicken', 'bacon', 'ranch', 'cheesesteak', '...",Philly Fresh Cheesesteaks (541-B Graymont Ave)
3,"['chicken', 'bacon', 'artichoke', '-', 'keto',...","['chicken', 'bacon', 'artichoke', 'keto', 'fri...",Papa Murphy's (1580 Montgomery Highway)
4,"['full', 'sausage', '(', '2', 'pcs', ')', 'wit...","['full', 'sausage', 'pc', 'egg', 'piece', 'dou...",Nelson Brothers Cafe (17th St N)
...,...,...,...
60922,"['breakfast', 'sampler', 'two', 'eggs', '*', '...","['breakfast', 'sampler', 'two', 'egg', 'way', ...",IHOP (4321 N 10th St)
60923,"['ready', 'rabbit', 'blue', 'curacao', 'non', ...","['ready', 'rabbit', 'blue', 'curacao', 'non', ...",Feldman's - N 10th Street
60924,"['heineken', 'non-alcoholic', '0.0', '6x', '11...","['heineken', 'can', 'master', 'brewer', 'start...",Feldman's - N 23rd St


In [7]:
input_str =  'carne asada burrito'

best_match = ''
best_score = 0

for i in range(0,60926):
    sc = fuzz.token_set_ratio( input_str, df['menu_final'][i])
    if sc > best_score:                    
        best_score = sc
        best_match = i

print(best_match)
print(best_score)

26
100


In [8]:
df['menu_final'][best_match]

"['build', 'burrito', 'two', 'scramble', 'egg', 'choice', 'meat', 'topping', 'wrap', 'fresh', 'tortilla', 'basic', 'breakfast', 'burrito', 'two', 'scramble', 'egg', 'breakfast', 'potato', 'melt', 'cheese', 'wrap', 'fresh', 'flour', 'tortilla', 'bacon', 'breakfast', 'burrito', 'two', 'scramble', 'egg', 'crispy', 'bacon', 'breakfast', 'potato', 'melt', 'cheese', 'wrap', 'fresh', 'flour', 'tortilla', 'carne', 'asada', 'breakfast', 'burrito', 'two', 'scramble', 'egg', 'carne', 'asada', 'breakfast', 'potato', 'melt', 'cheese', 'wrap', 'fresh', 'flour', 'tortilla', 'sausage', 'breakfast', 'burrito', 'two', 'scramble', 'egg', 'sausage', 'breakfast', 'potato', 'melt', 'cheese', 'wrap', 'fresh', 'flour', 'tortilla', 'spicy', 'hot', 'breakfast', 'burrito', 'two', 'scramble', 'egg', 'jalapeno', 'breakfast', 'potato', 'sriracha', 'melt', 'cheese', 'wrap', 'fresh', 'flour', 'tortilla', 'everything', 'kitchen', 'sink', 'breakfast', 'burrito', 'two', 'scramble', 'egg', 'bacon', 'carne', 'asada', 'bre

In [9]:
merged_df['menu'][random.randint(0, 60900)]

'[\'#\', \'1\', \'whataburger®\', \'whatameal®\', \'what\', "\'s", \'on\', \'it\', \':\', \'large\', \'bun\', \'(\', \'5\', "\'\'", \')\', \',\', \'large\', \'beef\', \'patty\', \'(\', \'5\', "\'\'", \')\', \'(\', \'1\', \')\', \',\', \'tomato\', \'(\', \'regular\', \')\', \',\', \'lettuce\', \'(\', \'regular\', \')\', \',\', \'pickles\', \'(\', \'regular\', \')\', \',\', \'diced\', \'onions\', \'(\', \'regular\', \')\', \',\', \'mustard\', \'(\', \'regular\', \')\', \'your\', \'whatameal®includes\', \':\', \'comes\', \'with\', \'medium\', \'fries\', \'and\', \'a\', \'medium\', \'drink\', \'.\', \'#\', \'2\', \'double\', \'meat\', \'whataburger®\', \'whatameal®\', \'what\', "\'s", \'on\', \'it\', \':\', \'large\', \'bun\', \'(\', \'5\', "\'\'", \')\', \',\', \'large\', \'beef\', \'patty\', \'(\', \'5\', "\'\'", \')\', \'(\', \'2\', \')\', \',\', \'tomato\', \'(\', \'regular\', \')\', \',\', \'lettuce\', \'(\', \'regular\', \')\', \',\', \'pickles\', \'(\', \'regular\', \')\', \',\', \'

In [10]:
merged_df['restaurant_name'][10412]

'Pure Gold by Qdoba (100 ANDOVER PARK WEST)'

In [11]:
menu_orders = [
    'carne asada breakfast burrito', 
    'bacon egg and cheese bagel', 
    'chile relleno stuffed with cheese and beef', 
    '8 pcs chicken meal with biscuits mashed potatoes and gravy',
    'tres leches cake and chocolate chip cookie',
    '3-cheese quesadilla with protein guacamole and sour cream',
    'chicken enchiladas with rice and beans plate',
    'baconator with 4 pc spicy chicken nuggets',
    'strawberry banana waffle with whipped cream and syrup',
    'pork dumpling with szechuan chili oil and fried rice',
    'cookie dough with rainbow sprinkles',
    'chili hot dog with chili cheese',
    'smoked wings with nachos with jalapenos ranch and pork',
    'wet cat food variety pack',
    'Romaine, iceberg, Roma tomatoes, cucumbers, carrots, and croutons',
    'Tempura shrimp and tempura veggies served over rice',
    'brownie covered with marshmallows and drizzled with caramel',
    'soup with prawns, calamari, pork, fish ball, egg',
    'Deep Fried Tofu Served sweet and sour',
    'Pulled tandoori chicken with tangy tomato gravy',
    'Peruvian style sirloin steak with onions and tomatoes with rice',
    'apple slices cripsy and juicy',
    'Greek Halal lamb and beef',
    'Sweet Onion Chicken Teriyaki sub',
    'chicken kabobs seasoned with spicy BBQ Sauce',
    'Shrimp cooked in hot sauce and rice',
    'spicy yellowtail roll with unagi sauce',
    'New York Cheesecake soft serve',
    'Grilled falafel wrapp pita bread',
    'Mini Pepperoni Pizza Bagels',
    'Tuna Salad Sandwich wheat bread',
    'Small Kibble Dry Dog Food with Real Chicken',
    'Salmon Livornese with shrimp and olives',
    'mahi-mahi tacos with cilantro chipotle mayonnaise',
    'Meatball Marinara Footlong with Parmesan cheese',
    'Salvadoran Style French Fries',
    'Sweet Vanilla Cream Iced Coffee',
    'Grilled Salmon with rémoulade sauce',
    'Veal Marsala Sautéed and topped with mushrooms',
    'Dragon Roll with Shrimp tempura eel and avocado',
    'Chicken Parmasan with Spaghetti and salad',
    'Toasted Cheddar Chalupa with grilled chicken',
    'Beef Brisket Soup with meat and vegetables  ',
    'breakfast sandwich with sausage bacon and ham',
    'Chicken Pork and Cheese Pupusas',
    'vegan Gluten-Free cuban black beans',
    'Greek Yogurt Parfait with fresh berries',
    'Mexican style Ribs with rice guacamole pico de gallo and tortillas',
    'Tomato Basil Bisque with cream basil and garlic',
    'Hibachi Filet Mignon with Japanese soy and teriyaki sauce'
]

In [12]:
gpt_orders = [
    "Pepperoni pizza with extra cheese, please.",
    "Grilled salmon with steamed veggies on the side.",
    "Chicken Caesar salad, dressing on the side.",
    "Beef burger with fries, well-done please.",
    "Veggie stir-fry with tofu and brown rice.",
    "Spaghetti carbonara with garlic bread.",
    "Shrimp tacos with avocado and salsa.",
    "Margherita flatbread pizza, light on sauce.",
    "Sushi combo with miso soup and salad.",
    "Turkey club sandwich with sweet potato fries.",
    "Pad Thai noodles, extra spicy if possible.",
    "Falafel wrap with hummus and tabbouleh.",
    "BBQ ribs with coleslaw and cornbread.",
    "Quinoa salad with grilled chicken and feta.",
    "Beef fajitas with peppers and onions.",
    "Mushroom risotto with Parmesan cheese.",
    "Chicken tikka masala with naan bread.",
    "Gyro platter with tzatziki sauce and fries.",
    "Veggie burger with avocado and sprouts.",
    "Pho with rare beef and fresh herbs.",
    "Lobster roll with melted butter on the side.",
    "Chicken teriyaki bowl with steamed rice.",
    "Caprese salad with ripe tomatoes and basil.",
    "Fish and chips with tartar sauce, please.",
    "Bruschetta with tomatoes, basil, and balsamic glaze.",
    "Cheeseburger deluxe with onion rings and cola.",
    "Caesar wrap with grilled chicken and parmesan.",
    "Vegetable curry with basmati rice and naan.",
    "Pulled pork sandwich with coleslaw on top.",
    "Sashimi platter with wasabi and soy sauce.",
    "Beef burrito bowl with guacamole and salsa.",
    "Margherita pizza with a thin, crispy crust.",
    "Chicken noodle soup with a side of crackers.",
    "Tofu pad see ew with broccoli and carrots.",
    "Hawaiian poke bowl with fresh tuna and rice.",
    "Philly cheesesteak with peppers and provolone.",
    "Spinach and feta omelette with home fries.",
    "Falafel plate with hummus and pita bread.",
    "Beef stroganoff with egg noodles and sour cream.",
    "Avocado toast with poached eggs and arugula.",
    "Buffalo wings with celery sticks and blue cheese.",
    "Bibimbap with assorted vegetables and gochujang.",
    "Margarita pizza with basil and cherry tomatoes.",
    "Chicken quesadilla with sour cream and salsa.",
    "Cobb salad with grilled chicken and ranch dressing.",
    "Veggie sushi roll with avocado and cucumber.",
    "Beef pho with bean sprouts and lime wedges.",
    "Grilled cheese sandwich with tomato soup.",
    "Chicken alfredo pasta with garlic bread.",
    "Greek salad with olives, cucumbers, and feta."
]


In [13]:
# gpt_match = []
# gpt_match_score = []
# for i in gpt_orders:
#     input_str =  i

#     best_match = ''
#     best_score = 0

#     for j in range(0,60926):
#         sc = fuzz.token_set_ratio( input_str, df['menu_final'][j])
#         if sc > best_score:                    
#             best_score = sc
#             best_match = j
#             gpt_match.append(best_match)
#             gpt_match_score.append(best_score)

#     print(best_match, " " , best_score)

In [14]:
# menu_match = []
# menu_match_score = []
# for i in menu_orders:
#     input_str =  i

#     best_match = ''
#     best_score = 0

#     for j in range(0,60926):
#         sc = fuzz.token_set_ratio( input_str, df['menu_final'][j])
#         if sc > best_score:                    
#             best_score = sc
#             best_match = j
#             menu_match.append(best_match)
#             menu_match_score.append(best_score)

#     print(best_match, " " , best_score)

In [15]:
# data = {
#     'search': menu_orders + gpt_orders,
#     'result': menu_match + gpt_match,
#     'score': menu_match_score + gpt_match_score
# }

# ld_data = pd.DataFrame(data)

# ld_data 

In [16]:
# ld_data.to_csv('ld_results.csv', index=False)
ld_data = pd.read_csv('ld_results.csv')
ld_data

,search,result,score
0,carne asada breakfast burrito,244,93
1,bacon egg and cheese bagel,39234,85
2,chile relleno stuffed with cheese and beef,3984,100
3,8 pcs chicken meal with biscuits mashed potato...,54465,76
4,tres leches cake and chocolate chip cookie,349,87
...,...,...,...
95,Veggie sushi roll with avocado and cucumber.,2554,91
96,Beef pho with bean sprouts and lime wedges.,33,80
97,Grilled cheese sandwich with tomato soup.,9427,93
98,Chicken alfredo pasta with garlic bread.,143,88


In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier

# Example data
X_train_text = merged_df.menu_final
y_train =  merged_df.index

# Convert text data to numerical features
vectorizer = TfidfVectorizer()
X_train_features = vectorizer.fit_transform(X_train_text)

# Initialize KNN classifier
knn = KNeighborsClassifier(n_neighbors=1)

# Train the classifier
knn.fit(X_train_features, y_train)


KNeighborsClassifier(n_neighbors=1)

In [40]:
# New data point to classify
X_new_text = 'carne asada breakfast burrito'

# Preprocess and convert new data to numerical features
X_new_preprocessed = lemmatizer.lemmatize(X_new_text)
X_new_features = vectorizer.transform([X_new_preprocessed])

# Predict the label for the new data point
best_match = knn.predict(X_new_features)

print("Best match:", best_match)


Best match: [42528]


In [41]:
merged_df['menu_final'][best_match]

restaurant_id
42528    ['ultimate', 'breakfast', 'burrito', 'egg', 'c...
Name: menu_final, dtype: object

In [42]:
merged_df['restaurant_name'][best_match]

restaurant_id
42528    Daydream Breakfast Burritos
Name: restaurant_name, dtype: object

In [43]:
knn_result = []

for i in range(0,100):
    # New data point to classify
    X_new_text = ld_data['search'][i]

    # Preprocess and convert new data to numerical features
    X_new_preprocessed = lemmatizer.lemmatize(X_new_text)
    X_new_features = vectorizer.transform([X_new_preprocessed])

    # Predict the label for the new data point
    best_match = knn.predict(X_new_features)
    knn_result.append(best_match)

    print("Best match:", best_match)
    

Best match: [42528]
Best match: [48514]
Best match: [30112]
Best match: [30112]
Best match: [30112]
Best match: [55362]
Best match: [30112]
Best match: [30112]
Best match: [49975]
Best match: [30112]
Best match: [30112]
Best match: [40275]
Best match: [30112]
Best match: [30112]
Best match: [30112]
Best match: [30112]
Best match: [30112]
Best match: [30112]
Best match: [30112]
Best match: [30112]
Best match: [30112]
Best match: [30112]
Best match: [26684]
Best match: [19625]
Best match: [30112]
Best match: [33986]
Best match: [30112]
Best match: [60729]
Best match: [16328]
Best match: [30112]
Best match: [30112]
Best match: [30112]
Best match: [30112]
Best match: [579]
Best match: [30112]
Best match: [55158]
Best match: [46219]
Best match: [30112]
Best match: [30112]
Best match: [29637]
Best match: [30112]
Best match: [30112]
Best match: [30112]
Best match: [55272]
Best match: [49966]
Best match: [30112]
Best match: [32337]
Best match: [30112]
Best match: [30112]
Best match: [17515]
Be

In [22]:
from sklearn.metrics import accuracy_score
ac = accuracy_score(knn_result,ld_data.result)
ac

0.0

In [44]:
ld_avg_sc = np.mean(ld_data.score)
ld_avg_sc

87.83

In [45]:
knn_match_score = []

for i in range(0,100):
    # New data point to classify
    X_new_text = ld_data['search'][i]

    # Preprocess and convert new data to numerical features
    X_new_preprocessed = lemmatizer.lemmatize(X_new_text)

    # Predict the label for the new data point
    sc = fuzz.token_set_ratio( X_new_preprocessed, merged_df['menu_final'][knn_result[i]])
    knn_match_score.append(sc)

    print( i , " score:", sc)
    

0  score: 74
1  score: 63
2  score: 24
3  score: 30
4  score: 33
5  score: 26
6  score: 7
7  score: 18
8  score: 32
9  score: 6
10  score: 19
11  score: 44
12  score: 25
13  score: 11
14  score: 6
15  score: 31
16  score: 24
17  score: 22
18  score: 7
19  score: 13
20  score: 6
21  score: 8
22  score: 57
23  score: 33
24  score: 28
25  score: 25
26  score: 14
27  score: 78
28  score: 33
29  score: 16
30  score: 18
31  score: 26
32  score: 23
33  score: 25
34  score: 21
35  score: 31
36  score: 52
37  score: 24
38  score: 17
39  score: 28
40  score: 7
41  score: 20
42  score: 27
43  score: 33
44  score: 18
45  score: 12
46  score: 27
47  score: 19
48  score: 7
49  score: 52
50  score: 40
51  score: 29
52  score: 45
53  score: 23
54  score: 15
55  score: 39
56  score: 7
57  score: 22
58  score: 24
59  score: 31
60  score: 14
61  score: 7
62  score: 40
63  score: 7
64  score: 7
65  score: 35
66  score: 50
67  score: 30
68  score: 7
69  score: 38
70  score: 44
71  score: 58
72  score: 22
7

In [46]:
knn_avg_sc = np.mean(knn_match_score)
knn_avg_sc

27.16

In [23]:
# from sklearn.svm import SVC
# svm = SVC(kernel='linear')

# # Train the classifier
# svm.fit(X_train_features, y_train)


In [24]:
# # New data point to classify
# X_new_text = 'carne asada breakfast burrito'

# # Convert new data to numerical features
# X_new_features = vectorizer.transform([X_new_text])

# # Predict the label for the new data point
# best_match = svm.predict(X_new_features)

# print("Best match:", best_match)

In [47]:
ld_data['search'][0]

'carne asada breakfast burrito'

In [53]:
merged_df['menu_final'][ld_data['result'][0]]

"['snicker', 'chocolate', 'candy', 'bar', 'fun', 'size', 'oz', 'load', 'roasted', 'peanut', 'nougat', 'caramel', 'milk', 'chocolate', 'snicker', 'chocolate', 'candy', 'satisfies', 'monday', 'night', 'football', 'hunger', 'super', 'bowl', 'tailgate', 'fantasy', 'football', 'snicker', 'chocolate', 'candy', 'bar', 'sure', 'please', 'friend', 'family', 'coworkers', 'stock', 'snicker', 'fun', 'size', 'chocolate', 'fill', 'autumn', 'candy', 'dish', 'hand', 'hunger', 'strike', 'next', 'official', 'sponsor', 'nfl', 'snicker', 'carry', 'sunday', 'night', 'monday', 'night', 'thursday', 'night', 'live', 'kit', 'kat', 'snack', 'size', 'wafer', 'bar', 'milk', 'chocolate', 'oz', 'scream', 'delicious', 'combination', 'smooth', 'milk', 'chocolate', 'light', 'crispy', 'wafer', 'plenty', 'snack', 'size', 'kit', 'kat', 'wafer', 'bar', 'leave', 'snack', 'baking', 'stock', 'candy', 'bowl', 'perfect', 'halloween', 'everyday', 'snacking', 'delicious', 'edible', 'party', 'decor', 'favor', 'layer', 'crisp', 'w

In [49]:
merged_df['menu_final'][knn_result[0]]

restaurant_id
42528    ['ultimate', 'breakfast', 'burrito', 'egg', 'c...
Name: menu_final, dtype: object